In [2]:
import numpy as np

# Markov Chains

A Markov Chain is one that says the probability of a state at time $t+1$ depends only on the state at time $t$ (called $i_t$) and does *not* depend on the chain of states passed through up until that point.

There is also a further assumption that the probability of a given transition is independent of time $t$-i.e. you are not more likely to go from A to B after 10 mins than after 5.
 $P(X_{t+1} =j|X_t=i)$ is independent of $t$
 
 The shorthand for this probability expression (the p of going to j in the next step given you are at i now) is $p_{ij}$
 
 A $p_{ij}$ value is known as the **transition probability**
 
 As  $p_{ij}$ does not change over time, it can be considered **stationary**. This is called the **Stationarity Assumption**.
 
 ### $q_i$
 Markov chains also require us to define the probability that a chain is in a given state, $i$ at time 0 (i.e. when $t=0$). In other words, 
 $$P(X_0 = i) = q_i$$
 
 
 A list of starting probability values $q_i$ can be expressed as a vector, called $\textbf{q}$ i.e.
 $$ \textbf{q} = [q_1 q_2 ... q_s] $$
 This is called the **initial probability distribution**
 Note $s$ is the number of states that can be held
 
 ### Transition probability matrix
 The **Transition Probability Matrix** is called $P$. It holds the mapping of all $p_{ij}$ values.
 As effectively, everything has to have a state, the sum of each row should be one. Specifically:
 $$\sum_{j=1}^{j=s} p_{ij} =1 $$
 
 All entries, being probabilities, should also be non-negative.
 
 ### n-step Transition Probabilities
A question of interest is that given a Markov chain in state $i$ at time $m$, what is the probability that the Markov chain will be in state $j$ after $n$ more periods?
Given the Stationarity Assumption, this should be independent of the value of $m$ and so
$$P(X_{m+n}=j | X_m =i)       =     P(X_n =j| X_0 =i) $$

This is called $P_{ij}(n)$, the **n-step probability** of a transition from state i to state j.

Obviously, where $n$=1, i.e. only one step, then this is the normal **transition probability** i.e. $P_{ij}(1) = p_{ij}$

If n=2, then we have to go via some intermediary state(s), expressed as k. So it would be $p_{ik} * p_{kj}$ - the probability of going from i to k, and then k to j.

Clearly there might be multiple different states, $k$ that could be travelled through to end up at the destination, $j$. (literally put, I can go A>>B>>D or A>>C>>D).
So we can say:
$$P_{ij}(2) = \sum_{k=1}^{k=s} p_{ik}.p_{kj}$$


All $p_{ik}$ values are held in row $i$ of the transition matrix, $P$. (i.e. all probabilities of a transition *from* state $i$.
Similarly, all probabilities of transititioning *to* a state $j$ are held in the *column* $j$ of the matrix, $P$.

So to get $P_{ij}(2)$, we just need the **scalar product** of row $i$ and column $j$ (remembering matrix algebra).

This resolves as the $ij$th element of the matrix $P^2$. Extending this further, it can be shown that

$P_{ij}(n) = ij$th element of $P^n$


In [ ]:
## Imagine a matrix where A>>A=90% ,A>B=10%, B>A=20% and B>B=80%.
P=np.array([[0.9,0.1],[0.2,0.8]])

## To get the likelihood of going from B to A (state 2 to state 1) after two steps we need element 2,1 of P^2.
print(np.matmul(P,P))
## Since we are specifically squaring it, we can use a power function too, rather than multiplying matrices
print(np.linalg.matrix_power(P,2))

## Overall it is:
#(B>>A * A>>A) + (B>>B * B>>A) i.e. (.2*.9)=.18 + (.8*.2)=.16 =.34 

## This returns 0.34 in position 2,1 (which is 1,0 in the 0-indexed world of Python)
np.linalg.matrix_power(P,2)[1,0]
## NOTE, use matmul (matrix multiply) to get the scalar product, not a simple "np.multiply"- which just multiplies ii with ii etc. And would return .2 ^2 =0.04

In [ ]:
#Similarly the probability of going from state 1 to state 1 in three steps is
np.linalg.matrix_power(P,3)[0,0]
#0.781

### Unknown starting points

We don't often know the exact starting point. For example, we don't know that we're currently in position $i$

In this case, the vector of starting positions  $\textbf{q}$ is needed- which lists the likelihood of being in each of the states. 
In examples above, we know we are definitely in e.g. state B and transitioning to state A. If we are uncertain of where we are starting, then all we can provide is a probability of ending up in state A, given the uncertain starting positions, and the probability of taking each route to A.

Effectively, if we want to know the likelihood of being in state A after two transitions then it needs:
P(A>>A transition) * P(starting in A)     +      P(B>>A transition)* P(starting in B)

This is in contrast to knowing we're starting in position B and transitioning to A in 2 steps, whereby it would simply be P(B>>A transition) * 1.

Mathematically this is
$$P(being\ in\ state\ j\ at\ time\ n) = \sum_{i=1}^{i=s} P(state\ is\ originally\ i) * P( going\ from\ i\ to\ j\ in\ n\ transitions)$$
$$ = \sum_{i=1}^{i=s} q_i P_{ij}(n) = \textbf{q}(column\ j\ of\ P^n) $$


In [ ]:
## If we use the above matrix, P. The three step matrix is P^3
print(np.linalg.matrix_power(P,3))

## If we want to know the likelihood of ending up in state 1 in three steps from now, we need the distribution of starting states
# Effectively, we need the likelihood of each state, as of now. Let's imagine there's a 60% chance of state 1 and 40% of state 2
q=np.array([.6,.4])
print(q)

#From the above, we want column j of P^3. So we want column 1 (or 0, for 0-indexing)
col=np.linalg.matrix_power(P,3)[:,0]
print(col)

print("Probability of being in state 1 after 3 steps is {:.1%}".format(np.matmul(q,col)))



## Classification of states

A **path** from $i$ to $j$ is a sequence of transistions that starts in $i$ and ends in $j$, where each transition in the sequence has a positive probability of occurring. I.e. it is a feasible route between states $i$ and $j$.

A state, $j$ is **reachable** from state $i$ if there is a path leading from $i$ to $j$.

Two states are said to **communicate** if they are *reachable* from each other, i.e. $i$ is *reachable* from $j$ and also $j$ is *reachable* from $i$. This isn't always the case, as paths don't always flow both ways.

A set of states $S$ is a **closed set** if no state outside of $S$ is *reachable* from any state in $S$. I.e. once you enter $S$- you can't leave and only bounce around the other states within the set $S$.

An **absorbing state** is one where $p_{ii}=1$ i.e. once you enter the state you cannot leave it (e.g. death!). An absorbing state is a *closed set* with only one state in the set, $S$.

A state $i$ is a **transient state** if there is any state,$j$ which is reachable from $i$ but $i$ is not reachable from $j$. So a state is transient if there is a least one way whereby, once you leave $i$, you cannot return.
If a state is not *transient* it is instead **recurrent** - i.e. it is always possible to return to it.

A state is **periodic** if all paths leading from $i$ back to $i$ are some multiple of a value $k$ where $k>1$. I.e. A>>B>>A has a k value of 2. If a recurrent state can be returned to via a number of different length paths, it is **aperiodic**.


An **ergodic** chain is one whereby all states are *recurrent, aperiodic and communicate* i.e. you can get from any state to any state infinitely. There is no absorption or periodicity. *In a practical sense, I think that this would mean a matrix has no 0 or 1 values.*



## Steady State Probabilities
Over time, after many transitions, the probalities can tend to settle down. Above there is a 64% chance of position 1 after 3 steps. After 11 steps it converges on about 67%.

Importantly, these probabilities don't depend on the initial starting state. Eventually, for ergodic chains there exists a vector of probabilities $\pi= [\pi_1 \pi_2 ... \pi_s] $ where e.g. $\pi_1$ is the probability of reaching state 1 is eventually, irrespective of the starting position, so e.g $p_{11}= \pi_1 = p_{21}$ etc. specifically:
$$ lim_{n\to\infty} P_{ij}(n) = \pi_j$$


The vector $\pi$ is called the **steady state distribution** or **equilibrium distribution**. Effectively it represents for states $1$ to $n$ the probability of ending up in each state (in an *ergodic* chain that is!).


### Solving steady state probabilities

Broadly, we know that for situations where $n$ tends to infinity, $\pi_j$ is the vector of probabilities of ending up there. 
To calculate the position at $n+1$, we would ordinarily use the one-step transition matrix, $P$. But at $n+1$ where $n$ is infinity, the probability is also $\pi$ so we're left with the situation:

$$ \pi = \pi P $$

We also know that because everything must end up somewhere, the sum of the probabilities within the vector $\pi =1$ i.e.
$$\pi= [\pi_1 \pi_2 ... \pi_s] $$
and 
$$\pi_1 + \pi_2 + ... + \pi_s =1 $$


Let's now imagine $ 
   P=
  \left[ {\begin{array} {cc}
   0.9 & 0.1 \\
   0.2 & 0.8  \\
  \end{array} } \right]
$
as above. Substituting it into $ \pi = \pi P $
We get 
$$ [\pi_1 \pi_2]= [\pi_1 \pi_2].  \left[ {\begin{array} {cc}
   0.9 & 0.1 \\
   0.2 & 0.8  \\
  \end{array} } \right]
 $$

In this case, 
$$ \pi_1 = 0.9\pi_1 + 0.2\pi_2 $$
So subtracting $0.9\pi_1$ from both sides,
$$ 0.1\pi_1 = 0.2\pi_2 $$
Multiplying both sides by 5,
$$ 0.5\pi_1 = \pi_2 $$

Given that we also know from earlier, because the contents of vector $\pi$ add up to 1, that 
$$ 1 = \pi_1 + \pi_2 $$
We can sub in to get 
$$ 1= \pi_1 + 0.5\pi_1 = 1.5\pi_1 $$
$$1/1.5 = \pi_1 = 2/3$$
$$ 0.5\pi_1 = \pi_2$$
$$ \pi_2 = 1/3 $$

In this way, you can solve for the steady- state values of an ergodic matrix.


# Transient Analysis
There is no rule as to how quickly a steady state is reached. Typically if $P$ contains lots of values close to 0 or 1, steady states are reached quickly.
The behaviour of a Markov chain before the steady state is reached is called the **transient** or **short-run** behaviour.


## Mean first passage times

Mean first passage times describe the expected number of transitions to first reach state i from j in an *ergodic* (i.e. infinite) chain, given that we are currently in state $i$. 
$m_ij$ is called the **mean first passage time** from state $i$ to state $j$.

In effect, this would be:
- The probability of going in one step as $p_{ij}$
- The probability of going in two steps $\sum_{k!=j} p_{ik} . p_{kj}$
- This goes on infinitely for three steps, four steps, etc.

Of course, if the number of steps to go from $i$ to $j$ = $m_{ij}$ then the number of steps to go from $k$ to $j$ is $m_{kj}$

The *direct* route between $i$ and $j$ is one step long and occurs with the probability $p_{ij}$. All other indirect routes via an intermediate step $k$ are $1+m_{kj}$ long. The "1" is the step from $i$ to $k$ and $m_{kj}$ is the length of steps from $k$ to $j$.

The means the **mean first passage time** is a weighted average of the direct route of length 1 and all indirect routes of length $1+m_{kj}$, with weights being the probability of going that route ($p_{ij}$ and each $p_{ik}$ respectively.

So,
$$ m_{ij} = (1).p_{ij} + \sum_{k\neq j} (1+m_{kj}).p_{ik}$$


Given that $k \neq j$ and we sum across all $k$, plus $j$- that is the entire set of possible states that can be navigated to, which therefore $=1$.
Specifically therefore, $$p_{ij} + \sum_{k \neq j} p_{ik} =1$$


The first equation can be expanded,
$$ m_{ij} = (1).p_{ij} + \sum_{k\neq j} (1+m_{kj}).p_{ik}$$
$$ m_{ij} = (1).p_{ij} + \sum_{k\neq j} p_{ik} + \sum_{k\neq j} p_{ik}.m_{kj}$$
The second equation can be subbed into the first equation:
$$ m_{ij} = 1 + \sum_{k\neq j} p_{ik}.m_{kj}$$


Through some other unspecified logic, it can be shown (apparently) that 
$$ m_{ii} = 1/ \pi_i $$



# Absorbing chains
The stuff above is related to ergodic chains, where every state can talk to every state infinitely, and there are no absorbing parts of the chain.
However, often there are absorbing chains of interest- where eventually everything converges on one or more states.

In this example, there are five states (Junior, Senior, Partner, Leave as Non-Partner, Leave as Partner). The last two states are *absorbing*: once you leave, you leave. The other states are *transient*- you can get e.g. from Senior to Leave but not vice versa.

$$ 
   P=
  \left[ {\begin{array} {cc}
   0.8 & 0.15 & 0 & 0.05 & 0 \\
   0 & 0.7 & 0.2 & 0.1 & 0 \\
   0 & 0 & 0.95 & 0 & 0.05 \\
   0 & 0 & 0 & 1 & 0 \\
   0 & 0 & 0 & 0 & 1 \\
  \end{array} } \right]
$$

For an absorbing chain we want to know a few things:
- Does it start in a transient state?
- How many times do we expect to go through each state before absorption? i.e. how many periods do we expect to be in each transient state before absorption? 
- If a chain begins in a given transient state, what is the probability of ending up in each different absorbing state?


To figure it all out, we write out the transition matrix in a specific way. Overall, there are $s$ states, with $m$ representing the number of absorbing states, and so $s-m$ transient states. The standard format of the matrix $P$ is:

$$ 
   P=
  \left[ {\begin{array} {cc}
   Q & R \\
   0 & I \\
   \end{array} } \right]
$$

In this
- $Q$= the matrix of probabilities for movement between transient states - sized $s-m$ * $s-m$ 
- $R$= the matrix of probabilities for movement from transient to absorbing states - sized $s-m$ rows by $m$ columns
- $0$= the matrix of probabilities for movement from absorbing to transient states. As this is impossible, all values are 0. The matrix is $m$ rows and $s-m$ columns
- $I$ is the matrix of probabilities of moving from absorbing to absorbing states. As they are absorbing, all values are 1 on the diagonal, creating an "identity" matrix. The matrix is sized $m$ by $m$. 


In the above example,
$$ 
   Q=
  \left[ {\begin{array} {cc}
   0.8 & 0.15 & 0 \\
   0 & 0.7 & 0.2 \\
   0 & 0 & 0.95  \\
  \end{array} } \right]
$$
 and 

$$ 
   R=
  \left[ {\begin{array} {cc}
   0.05 & 0 \\
   0.1 & 0 \\
   0 & 0.05 \\
   \end{array} } \right]
$$


## Markov chain's fundamental matrix
The **Markov chain's fundamental matrix** is the matrix $(I-Q)^{-1}$.
It has some interesting properties that allow the above questions to be answered.

#### How many times do we expect to go through each state before absorption? i.e. how many periods do we expect to be in each transient state before absorption? 
If we start in a transient state $t_i$, then the number of times we visit a transient state $t_j$  is the $ij$th element of the matrix $(I-Q)^{-1}$.
#### If a chain begins in a given transient state, what is the probability of ending up in each different absorbing state?
If we start in a transient state $t_i$, then the probability that we will eventually be absorbed into a given absorbing state $a_j$ is the $ij$th element of the matrix $(I-Q)^{-1}. R$.




In [11]:
# For the example above:
Q = np.array([[0.8,0.15,0],[0,0.7,0.2],[0,0,0.95]])
R = np.array([[0.05,0],[0.1,0],[0,0.05]])

len_Q=len(Q) # Get the length of Q to identify the size that the identity matrix, I needs to be
I_minus_Q= np.subtract(np.identity(len_Q),Q) 
I_minus_Q

In [15]:
# Raising to power of -1 gives fundamental matrix
Fundamental_matrix=np.linalg.matrix_power(I_minus_Q,-1)
print(Fundamental_matrix)


In [17]:
# Multiplying by R
np.matmul(Fundamental_matrix,R)


In [32]:
# Imagine we have a churn matrix of month 0, 1,2 and churn

In [21]:
churn_p=np.array([[0,0.5,0,0.5],[0,0,0.7,0.3],[0,0,0,1],[0,0,0,1]])
churn_p

In [ ]:
# After 2 periods, the probability matrix would be this

In [28]:
after2=np.linalg.matrix_power(churn_p,2)
after2

In [ ]:
#multiply by the current distribution of the base (the starting vectors) and you get the eventual distribution, 
#and notably, the % who churn

In [33]:
np.matmul([0.5,0.5,0,0],after2)

In [34]:
# This could be used to work out the year-end position of the current base, where you'd multiply by number of months left in year
# To this, you'd also have to add each month's acqn cohort of trial starts by the number of months they had in the year left, with
# a starting vector of [1,0....0]

In [ ]:
# You could then model where the FTS targets leave you as a year-end subs number OR figure out what you need FTS to be to hit the right subs number
# To get the full churn curve, you'd have to model out months not yet observed using a decay function.

In [35]:
## Thoughts: for a churn model for the year we could have 12 cohorts- the month they joined.

#For each cohort, we can look at the state after 12-n months. I.e. cohort 0 we look at after 12 months, cohort 1 after 11 etc.



In [38]:
from scipy.optimize import curve_fit
def func(x, a, b, c):
    return a*np.exp(-b*x) + c

x = np.linspace(0,4,50)
y = func(x, 2.5, 1.3, 0.5)
yn = y + 0.2*np.random.normal(size=len(x))

popt, pcov = curve_fit(func, x, yn)

In [43]:
def new_func(x,m,c):
    return m*x+c

curve_fit(new_func,[0,1,2],[2,3,4])

In [ ]:
def new_func(x,m,c):
    return m*x+c

curve_fit(new_func,[0,1,2],[2,3,4])

In [ ]:
empty_p=np.empty((14,14))

for n in range(0,14):
    print(n)
    pij=0.2
    if n<13:
        empty_p[n,n+1]=pij #Probability of next state-- random number for now
    empty_p[n,13]=1-pij
    
empty_p